In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [3]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if label_visiprog[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label


In [4]:
accuracy_vsp = []
accuracy = []

for i in range(10):
    X_train, X_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
    train_test_split(raw_feature, label_material, label_visiprog, \
                     test_size=0.5, stratify=label_material)


    model_vsp = LFDA_VISIPROG()
    model_vsp.fit(X_train, Y_VSP_train)
    X_vsp_test = model_vsp.transform(X_test)
    res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
    accuracy_vsp.append(res_vsp['accuracy'])
    
    
    model = LFDA()
    model.fit(X_train, Y_mat_train)
    X_mat_test = model.transform(X_test)
    res = leave_one_sample_out(X_mat_test, Y_mat_test)
    accuracy.append(res['accuracy'])

In [5]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy})
display(df)

,material,visiprog
0,0.980557,0.974457
1,0.981700,0.976363
2,0.979032,0.975982
3,0.977888,0.969882
4,0.975219,0.973694
5,0.979794,0.973694
6,0.980175,0.973694
7,0.978269,0.974838
8,0.980175,0.970644
9,0.974457,0.971407


In [7]:
df.mean()

material    0.978727
visiprog    0.973465
dtype: float64